In [4]:
# conda activate /home/zhengweiguo/miniconda3/envs/hnns
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from binary_io import *
import json
import os
import argparse

argparse = argparse.ArgumentParser()

dataset = 'gist1m'
dataset = 'deep100m.base'
dataset = 'datacomp-image.base'
dataset = 'imagenet.base'
dataset = 'datacomp-text.base'
dataset = 'spacev100m.base'
dataset = 'wikipedia.base'
config = json.loads(open('config.json').read())
M, efs = config[dataset]["M"], config[dataset]["efs"]

threshold = 950
query_only = False
print(threshold, query_only)
dim = config[dataset]["dim"]
efc = 1000
ck_ts = 1000
k = 1000
topk = 0

data_prefix = '/data/disk1/liuchengjun/HNNS/sample/'
checkpoint_prefix = '/data/disk1/liuchengjun/HNNS/checkpoint/'
prefix = f'{dataset}.M_{M}.efc_{efc}.efs_{efs}.ck_ts_{ck_ts}.ncheck_100.recall@{k}'

train_feature = fvecs_read(f'{data_prefix}{prefix}.train_feats_nn.fvecs')
test_feature = fvecs_read(f'{data_prefix}{prefix}.test_feats_nn.fvecs')[:, :]
if query_only:
    train_feature = train_feature[:, :dim]
    test_feature = test_feature[:, :dim]

print(f'data_path: {data_prefix}{prefix}')
train_number_recall = ivecs_read(f'{data_prefix}{prefix}.train_label.ivecs')[:, 0]
train_comps = ivecs_read(f'{data_prefix}{prefix}.train_label.ivecs')[:, 1]
test_number_recall = ivecs_read(f'{data_prefix}{prefix}.test_label.ivecs')[:, 0]
test_comps = ivecs_read(f'{data_prefix}{prefix}.test_label.ivecs')[:, 1]

train_label_classification = train_number_recall < threshold
test_label_classification = test_number_recall < threshold
# train_label_classification = 1.000 - train_number_recall / 1000
# test_label_classification = 1.000 - test_number_recall / 1000
train_label_regression = np.log2(train_comps + 1)
test_label_regression = np.log2(test_comps + 1)

print(np.sum(test_label_classification), np.sum(train_label_classification))


df = pd.DataFrame()

offset = 0
feat_query = train_feature[:, offset: offset + dim]
query_cols = [f"query_{i}" for i in range(dim)]
df_query = pd.DataFrame(feat_query, columns=query_cols)
df = pd.concat([df, df_query], axis = 1)
offset += dim

feat_query = train_feature[:, offset: offset + topk * (topk - 1) // 2]
query_cols = [f"cross_{i}" for i in range(topk * (topk - 1) // 2)]
df_query = pd.DataFrame(feat_query, columns=query_cols)
df = pd.concat([df, df_query], axis = 1)
offset += topk * (topk - 1) // 2

if offset < len(train_feature[0]):
    feat_dist = train_feature[:, offset: offset + 100]
    dist_cols = [f"dist_{i}" for i in range(100)]
    df_dist = pd.DataFrame(feat_dist, columns=dist_cols)
    df = pd.concat([df, df_dist], axis = 1)
    offset += 100
if offset < len(train_feature[0]):
    feat_dist = train_feature[:, offset: offset + 10]
    degree_cols = [f"degree_{i}" for i in range(10)]
    df_degree = pd.DataFrame(feat_dist, columns=degree_cols)
    df = pd.concat([df, df_degree], axis = 1)
    offset += 10
if offset < len(train_feature[0]):
    feat_update = train_feature[:, offset: offset + 3]
    update_cols = [f"update_{i}" for i in range(3)]
    df_update = pd.DataFrame(feat_update, columns=update_cols)
    df = pd.concat([df, df_update], axis = 1)
    offset += 3

assert offset == len(train_feature[0])

df_number_recall = pd.DataFrame(train_number_recall.reshape(-1, 1), columns=["number_recall"])
df_comps = pd.DataFrame(train_comps.reshape(-1, 1), columns=["comps"])
df = pd.concat([df, df_number_recall, df_comps], axis = 1)

950 False
data_path: /data/disk1/liuchengjun/HNNS/sample/wikipedia.base.M_32.efc_1000.efs_1000.ck_ts_1000.ncheck_100.recall@1000
3415 175820


In [5]:
print(df['update_0'].describe())
print(df['update_1'].describe())
print(df['update_2'].describe())

df['update_0_bin'] = pd.cut(df['update_0'], bins=5, labels=False)
df['update_1_bin'] = pd.cut(df['update_1'], bins=5, labels=False)
df['update_2_bin'] = pd.cut(df['update_2'], bins=5, labels=False)

def get_min_max(df, column, target_column='number_recall'):
    # comps
    result_dict = {}
    for bucket in range(5):
        subset = df[df[column] == bucket][target_column]
        if subset.empty:
            result_dict[bucket] = (None, None)
        else:
            min_value = np.percentile(subset, 5)
            max_value = np.percentile(subset, 95)
            median = subset.median()
            # 还需要存每个桶的左右边界
            lb = df[df[column] == bucket][column[:-4]].min()
            rb = df[df[column] == bucket][column[:-4]].max()
            result_dict[bucket] = (min_value, max_value, median, lb, rb)
    return result_dict

result_dict_update_0 = get_min_max(df, 'update_0_bin')
result_dict_update_1 = get_min_max(df, 'update_1_bin')
result_dict_update_2 = get_min_max(df, 'update_2_bin')

print("根据update_0分成5个桶，每个桶的number_recall的最小值和最大值情况如下：")
for bucket, (min_val, max_val, median, lb, rb) in result_dict_update_0.items():
    print(f"桶{bucket}：最小值为{min_val}，最大值为{max_val}, 中位数为{median}, 左边界为{lb}, 右边界为{rb}")

print("根据update_1成5个桶，每个桶的number_recall的最小值和最大值情况如下：")
for bucket, (min_val, max_val, median, lb, rb) in result_dict_update_1.items():
    print(f"桶{bucket}：最小值为{min_val}，最大值为{max_val}, 中位数为{median}, 左边界为{lb}, 右边界为{rb}")

print("根据update_2成5个桶，每个桶的number_recall的最小值和最大值情况如下：")
for bucket, (min_val, max_val, median, lb, rb) in result_dict_update_2.items():
    print(f"桶{bucket}：最小值为{min_val}，最大值为{max_val}, 中位数为{median}, 左边界为{lb}, 右边界为{rb}")
    

count    500000.000000
mean          8.975452
std           3.121855
min           0.000000
25%           7.000000
50%           9.000000
75%          11.000000
max          28.000000
Name: update_0, dtype: float64
count    500000.000000
mean         23.260653
std          25.144327
min           0.000000
25%           5.000000
50%          15.000000
75%          32.000000
max         247.000000
Name: update_1, dtype: float64
count    500000.000000
mean        214.972977
std          54.556202
min          66.000000
25%         176.000000
50%         211.000000
75%         250.000000
max         525.000000
Name: update_2, dtype: float64
根据update_0分成5个桶，每个桶的number_recall的最小值和最大值情况如下：
桶0：最小值为820.0，最大值为996.0, 中位数为955.0, 左边界为0.0, 右边界为5.0
桶1：最小值为848.0，最大值为998.0, 中位数为966.0, 左边界为6.0, 右边界为11.0
桶2：最小值为882.0，最大值为999.0, 中位数为977.0, 左边界为12.0, 右边界为16.0
桶3：最小值为905.0，最大值为999.0, 中位数为984.0, 左边界为17.0, 右边界为22.0
桶4：最小值为936.4，最大值为999.3, 中位数为988.0, 左边界为23.0, 右边界为28.0
根据update_1成5个桶，每个桶的number_recall的最小值和最大值情